In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

#model
model_id ="facebook/opt-350m"

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

The model conversion technique is totally similar as the one presented in the [8 bit integration blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) - it is based on module replacement. If you print the model, you will see that most of the `nn.Linear` layers are replaced by `bnb.nn.Linear4bit` layers!

In [5]:
print(model)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
      (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (

In [6]:
text = "hello my name is"
device = "cuda:0"

inputs = tokenizer(text,return_tensors="pt").to(device)
outputs = model.generate(**inputs,max_new_tokens=20)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


In [7]:
len(outputs[0])

25

In [8]:
tokenizer.decode(outputs[0],skip_special_tokens=True)

'hello my name is jimmy and i am a newb to reddit. i have been on reddit for about a'

In [10]:
import torch
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4_bit_compute_dtype=torch.bfloat16
)

model_bf16 = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=quantization_config)
outputs = model.generate(**inputs,max_new_tokens=20)


In [11]:
tokenizer.decode(outputs[0],skip_special_tokens=True)

'hello my name is jimmy and i am a newb to reddit. i have been on reddit for about a'

In [12]:
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4"
)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=quantization_config)
outputs = model.generate(**inputs,max_new_tokens=20)
tokenizer.decode(outputs[0],skip_special_tokens=True)

'hello my name is jayden and i am a newbie to this game. i have been playing for a few'

In [13]:
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_double_quant=True
)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=quantization_config)
outputs = model.generate(**inputs,max_new_tokens=20)
tokenizer.decode(outputs[0],skip_special_tokens=True)

'hello my name is jimmy and i am a newb to reddit. i have been on reddit for about a'

In [14]:
from transformers.onnx.features import AutoModel
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_double_quant=True
)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=quantization_config)
outputs = model.generate(**inputs,max_new_tokens=20)
tokenizer.decode(outputs[0],skip_special_tokens=True)

'hello my name is jayden and i am a newbie to this game. i have been playing for a few'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neox-20b"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_double_quant=True
)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=quantization_config,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [18]:
model.hf_device_map

{'': 0}

In [ ]:
text = "hello my name is"
device="cuda:0"
inputs = tokenizer(text,return_tensors="pt").to(device)
outputs = model.generate(**inputs,max_new_tokens=20)


In [20]:
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

hello my name is john

A:

You can use the following regex:
/^(\w
